# feature_store_igr: Entendendo

## Bibliotecas

In [1]:
import os
import pandas as pd
import zipfile
from datetime import date
import dotenv
import boto3
from io import BytesIO

import warnings
warnings.filterwarnings("ignore")

import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

'pt_BR.utf8'

In [2]:
rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')
env = os.path.join(rootPath, '.env')
dotenv.load_dotenv(dotenv_path=env)

print("Iniciando carregamento dos dados")
zip_file = os.path.join(dataPath, 'rating_igr.zip')
z = zipfile.ZipFile(zip_file)

Iniciando carregamento dos dados


# Funções

In [3]:
def ler_bases_exportadas(nome_arquivo):
    z.extract(nome_arquivo)
    df = pd.read_csv(nome_arquivo, sep=',')
    os.remove(nome_arquivo)
    return df


# Transforma as chaves de tempo em data
def coleta_datas(df, left_on, nova_coluna_data):

    df = pd.merge(
        left=df, right=base_dim_datas,
        left_on=left_on, right_on='chave_tempo',
        how= 'left'
        ).rename(
            columns={"date": nova_coluna_data})
    return df


def contar_num_ocorr_por_cda(DF, VAR_ID):
    DF_num_ocorr_cda = DF[VAR_ID].value_counts().to_frame().reset_index()

    if VAR_ID == 'cda':
        dict_col = {'cda':'num_ocorr', 'index':'cda'}
    else:
        dict_col = {'id_pessoa':'num_ocorr', 'index':'id_pessoa'}
        
    DF_num_ocorr_cda = DF_num_ocorr_cda.rename(columns = dict_col)
    DF_num_ocorr_cda = DF_num_ocorr_cda.sort_values(by = 'num_ocorr', ascending = False)
    return DF_num_ocorr_cda


def formatar_moeda(valor):
    return locale.currency(valor, grouping=True)


def paste_intervalo(row):
    return "[" + str(row['inf']) + ", " + str(row['sup']) + ")"


def up_s3_files(dataframe, bucket_name, folder_name, file_name):
    csv_buffer = BytesIO()
    dataframe.to_csv(csv_buffer, sep=';', index=False)
    file_key_aws = folder_name + file_name
    s3_resource.Object(bucket_name, file_key_aws).put(Body=csv_buffer.getvalue())

# Ler DADOS

In [4]:
base_notas_fiscais = ler_bases_exportadas('emissao_notas2.csv')
#base_conjunta = ler_bases_exportadas('conjunta.csv')
base_conjunta = ler_bases_exportadas('conjunta_da_aberto2.csv')

#### base_conjunta

In [5]:
numcda = base_conjunta.groupby('cda')['da_aberto'].nunique().reset_index()
numcda = numcda.sort_values(by = "da_aberto", ascending= False)
numcda

,cda,da_aberto
1520998,a133b7ade34d61a37c8fa7a67092eaab1,2
980557,67f104d567655dff6b3a8e268c80aedf1,2
980606,67f26faccbe420fb7c8fa7a67092eaab1,2
980605,67f26c575d1654609fcacff532cf50c42,2
1500543,9f07122cab71749f6b3a8e268c80aedf1,2
...,...,...
832859,584f25a9a43fb3387c8fa7a67092eaab1,1
832858,584f256ac036644e2b46f673bf0c61252,1
832857,584f256ac036644e2b46f673bf0c61251,1
832856,584f1d3fc58f55709ceed47545e621dd2,1


In [6]:
base_conjunta[base_conjunta['cda'] == "a133b7ade34d61a37c8fa7a67092eaab1"]

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,valor_pago,valor_tot,vlr_tributo,vlr_taxa,...,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,idade_divida,quantidade_reparcelamento,da_aberto,endereco_existe
1621639,a133b7ade34d61a37c8fa7a67092eaab1,imovel,253ce2a61701484f,CASA,ATIVO,IPTU,933.82,933.82,933.82,0.0,...,1,0,0,0,8.0,8.0,6.0,0,0,1
1621640,a133b7ade34d61a37c8fa7a67092eaab1,imovel,253ce2a61701484f,CASA,ATIVO,IPTU,0.00,3881.78,3881.78,0.0,...,1,0,0,0,1.0,0.0,6.0,0,1,1


### Notas Fiscais

In [7]:
nf_num_ocorr_cda = base_notas_fiscais.groupby('id_pessoa').value_counts().to_frame().reset_index()
nf_num_ocorr_cda

,id_pessoa,qtd_notas_2anos,0
0,0000e13a7d8d7537,78,1
1,000143d53dd61d40,272,1
2,00018057fd2ff824,1,1
3,0001bb635cb6c379,4,1
4,0003a25ca713d631,30,1
...,...,...,...
43022,fffc2ed4460bead4,9,1
43023,fffc82154ad1ee43,3,1
43024,fffe9f81e524f143,37,1
43025,ffffc714021c7c44,318,1


# Métricas

In [8]:
#base_conjunta['data_divida'] = pd.to_datetime(base_conjunta['inscricao_divida'], infer_datetime_format=True)
#base_conjunta['ano_inscricao_da'] = base_conjunta['data_divida'].dt.year
base_conjunta.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'valor_pago', 'valor_tot', 'vlr_tributo', 'vlr_taxa',
       'competencia_divida', 'inscricao_divida', 'arrecadacao_divida',
       'ajuizamento_divida', 'edificacao', 'cpf_cnpj_existe', 'protesto',
       'ajuizamento', 'refis', 'deb_totais', 'deb_pagos', 'idade_divida',
       'quantidade_reparcelamento', 'da_aberto', 'endereco_existe'],
      dtype='object')

In [ ]:
#base_conjunta.drop_duplicates(subset = 'cda', inplace = True) # Garantia que não houve duplicatas de linhas

In [9]:
bs_conj_da_ab_0 = base_conjunta[base_conjunta['da_aberto'] == 0] # 1608610 com da_aberto == 0
bs_conj_da_ab_1 = base_conjunta[base_conjunta['da_aberto'] == 1]

In [10]:
bs_conj_da_ab_0.shape[0]

1609354

In [11]:
bs_conj_da_ab_1.shape[0]

967079

In [12]:
base_conjunta.shape[0]

2576433

In [13]:
bs_conj_da_ab_0[bs_conj_da_ab_0['cda'] == '8d1f5d34b36fe9dce52281a70553db462']

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,valor_pago,valor_tot,vlr_tributo,vlr_taxa,...,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,idade_divida,quantidade_reparcelamento,da_aberto,endereco_existe
1419990,8d1f5d34b36fe9dce52281a70553db462,mercantil,08d5af2c269cf4ba,RESTAURANTES E SIMILARES,INAPTO,ISS,760.65,11200.74,11200.74,0.0,...,1,0,0,0,6.0,4.0,6.0,2,0,0


In [14]:
bs_conj_da_ab_1[bs_conj_da_ab_1['cda'] == '8d1f5d34b36fe9dce52281a70553db462']

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,valor_pago,valor_tot,vlr_tributo,vlr_taxa,...,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,idade_divida,quantidade_reparcelamento,da_aberto,endereco_existe
1419991,8d1f5d34b36fe9dce52281a70553db462,mercantil,08d5af2c269cf4ba,RESTAURANTES E SIMILARES,INAPTO,ISS,0.0,6170.01,6170.01,0.0,...,1,0,0,0,1.0,0.0,3.0,2,1,0


In [15]:
base_conjunta[base_conjunta['cda'] == '8d1f5d34b36fe9dce52281a70553db462']

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,valor_pago,valor_tot,vlr_tributo,vlr_taxa,...,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,idade_divida,quantidade_reparcelamento,da_aberto,endereco_existe
1419990,8d1f5d34b36fe9dce52281a70553db462,mercantil,08d5af2c269cf4ba,RESTAURANTES E SIMILARES,INAPTO,ISS,760.65,11200.74,11200.74,0.0,...,1,0,0,0,6.0,4.0,6.0,2,0,0
1419991,8d1f5d34b36fe9dce52281a70553db462,mercantil,08d5af2c269cf4ba,RESTAURANTES E SIMILARES,INAPTO,ISS,0.00,6170.01,6170.01,0.0,...,1,0,0,0,1.0,0.0,3.0,2,1,0


## Outra CDA

In [16]:
base_conjunta[base_conjunta['cda'] == '0000a43dae672b166b3a8e268c80aedf1']

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,valor_pago,valor_tot,vlr_tributo,vlr_taxa,...,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,idade_divida,quantidade_reparcelamento,da_aberto,endereco_existe
22,0000a43dae672b166b3a8e268c80aedf1,imovel,f5fe60dbb517e68a,APARTAMENTO,ATIVO,IPTU,1692.36,1692.36,1692.36,0.0,...,1,1,0,0,5.0,5.0,1.0,0,0,1
23,0000a43dae672b166b3a8e268c80aedf1,imovel,f5fe60dbb517e68a,APARTAMENTO,ATIVO,IPTU,0.00,2108.87,2108.87,0.0,...,1,1,0,0,1.0,0.0,1.0,0,1,1


In [17]:
bs_conj_da_ab_0[bs_conj_da_ab_0['cda'] == '0000a43dae672b166b3a8e268c80aedf1']

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,valor_pago,valor_tot,vlr_tributo,vlr_taxa,...,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,idade_divida,quantidade_reparcelamento,da_aberto,endereco_existe
22,0000a43dae672b166b3a8e268c80aedf1,imovel,f5fe60dbb517e68a,APARTAMENTO,ATIVO,IPTU,1692.36,1692.36,1692.36,0.0,...,1,1,0,0,5.0,5.0,1.0,0,0,1


In [18]:
bs_conj_da_ab_1[bs_conj_da_ab_1['cda'] == '0000a43dae672b166b3a8e268c80aedf1']

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,valor_pago,valor_tot,vlr_tributo,vlr_taxa,...,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,idade_divida,quantidade_reparcelamento,da_aberto,endereco_existe
23,0000a43dae672b166b3a8e268c80aedf1,imovel,f5fe60dbb517e68a,APARTAMENTO,ATIVO,IPTU,0.0,2108.87,2108.87,0.0,...,1,1,0,0,1.0,0.0,1.0,0,1,1


### Fechado/Treino

In [19]:
formatar_moeda(bs_conj_da_ab_0['valor_tot'].sum())

'R$ 11.763.979.699,63'

In [20]:
formatar_moeda(bs_conj_da_ab_0['valor_pago'].sum())

'R$ 2.458.003.557,95'

### Aberto/Estoque/Predição

In [21]:
formatar_moeda(bs_conj_da_ab_1['valor_tot'].sum())

'R$ 13.863.337.109,58'

In [22]:
formatar_moeda(bs_conj_da_ab_1['valor_pago'].sum())

'R$ 55.001,68'

# 02) Manipulando Dados

In [23]:
def criar_obj_valor_tot_pago_aberto(base_conjunta):
    valor_tot = base_conjunta.groupby(['cda', 'tipo_divida', 'id_pessoa'])['valor_tot'].sum().to_frame().reset_index()
    valor_pago = base_conjunta.groupby(['cda', 'tipo_divida', 'id_pessoa'])['valor_pago'].sum().to_frame().reset_index()

    # O que está em aberto
    valor_aberto_tot = base_conjunta[base_conjunta['da_aberto'] == 1].groupby(['cda', 'tipo_divida', 'id_pessoa'])['valor_tot'].sum().to_frame().reset_index()
    valor_aberto_pg = base_conjunta[base_conjunta['da_aberto'] == 1].groupby(['cda', 'tipo_divida', 'id_pessoa'])['valor_pago'].sum().to_frame().reset_index()
    valor_aberto = pd.merge(valor_aberto_tot, valor_aberto_pg, on = ['cda', 'tipo_divida', 'id_pessoa'], how = "left")
    valor_aberto['valor_aberto'] = valor_aberto['valor_tot'] - valor_aberto['valor_pago']
    valor_aberto.drop(columns = ['valor_tot', 'valor_pago'], inplace = True)

    # Merge de valor_tot & valor_pago
    valor_tot_pago = pd.merge(valor_tot, valor_pago, on = ['cda', 'tipo_divida', 'id_pessoa'], how = "left")
    # Merge de (valor_tot e valor_pago) & valor_aberto
    valor_tot_pago_aberto = pd.merge(valor_tot_pago, valor_aberto, on = ['cda', 'tipo_divida', 'id_pessoa'], how = "left")

    # O que a gente esperava receber: dif_tot_pago
    valor_tot_pago_aberto['dif_tot_pago'] = valor_tot_pago_aberto['valor_tot'] - valor_tot_pago_aberto['valor_pago']
    # O quanto perdeu entre o que a gente esperava receber e o que foi efetivamente pago
    valor_tot_pago_aberto['dif_tot_pago_aberto'] = round(valor_tot_pago_aberto['dif_tot_pago'] - valor_tot_pago_aberto['valor_aberto'], 5)

    valor_tot_pago_aberto.sort_values(by = 'dif_tot_pago_aberto', ascending = False)

    return valor_tot_pago_aberto

In [24]:
valor_tot_pago_ab_0 = criar_obj_valor_tot_pago_aberto(bs_conj_da_ab_0)

valor_tot_pago_ab_1 = criar_obj_valor_tot_pago_aberto(bs_conj_da_ab_1)

# Renomeia colunas para nome mais adequados e filtra dataframe
colunas_nome = {
    'valor_tot': 'valor_total_da',
    'anos_idade_da': 'idade_divida'
}

valor_tot_pago_ab_0 = valor_tot_pago_ab_0.rename(columns = colunas_nome)

valor_tot_pago_ab_1 = valor_tot_pago_ab_1.rename(columns = colunas_nome)

In [25]:
valor_tot_pago_ab_0.shape[0]

1609354

In [26]:
valor_tot_pago_ab_1.shape[0]

967079

In [27]:
# Criando variável target Y
#valor_tot_pago_ab_0.loc[(valor_tot_pago_ab_0['valor_pago'].isna()) | (valor_tot_pago_ab_0['valor_pago'] == 0) , 'valor_pago'] = 1
valor_tot_pago_ab_0.loc[(valor_tot_pago_ab_0['valor_total_da'].isna()) | (valor_tot_pago_ab_0['valor_total_da'] == 0) , 'valor_total_da'] = 1

valor_tot_pago_ab_0['percentual_pago_cda'] = valor_tot_pago_ab_0['valor_pago'] / (valor_tot_pago_ab_0['valor_total_da'])

valor_tot_pago_ab_0

,cda,tipo_divida,id_pessoa,valor_total_da,valor_pago,valor_aberto,dif_tot_pago,dif_tot_pago_aberto,percentual_pago_cda
0,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,1278.97,0.00,NaN,1278.97,NaN,0.000000
1,0000331f601a73e52b46f673bf0c61251,imovel,870c08c252b25ad1,344.12,0.00,NaN,344.12,NaN,0.000000
2,000040eda866e3d19ceed47545e621dd1,imovel,9f3bac8718dac1fa,10779.87,5438.40,NaN,5341.47,NaN,0.504496
3,000040eda866e3d19ceed47545e621dd2,mercantil,2f8ae1b9606267b4,328.63,0.00,NaN,328.63,NaN,0.000000
4,0000521b64bf28c8e52281a70553db461,imovel,b5df816e1786f0a3,490.83,0.00,NaN,490.83,NaN,0.000000
...,...,...,...,...,...,...,...,...,...
1609349,ffffd79d8aa612ac9fcacff532cf50c41,imovel,c6b16a1235d987d3,26104.42,26104.42,NaN,0.00,NaN,1.000000
1609350,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,cc5afb41f6fbc367,1431.77,265.15,NaN,1166.62,NaN,0.185190
1609351,ffffd8af5760cada9fcacff532cf50c41,imovel,d411f54ad97d1f39,1728.13,0.00,NaN,1728.13,NaN,0.000000
1609352,ffffdb721b90282718565e26094f06ee2,mercantil,51d6043a4efa1ed1,240.77,0.00,NaN,240.77,NaN,0.000000


In [28]:
# Criando variável target Y
#valor_tot_pago_ab_1.loc[(valor_tot_pago_ab_1['valor_pago'].isna()) | (valor_tot_pago_ab_1['valor_pago'] == 0) , 'valor_pago'] = 1
valor_tot_pago_ab_1.loc[(valor_tot_pago_ab_1['valor_total_da'].isna()) | (valor_tot_pago_ab_1['valor_total_da'] == 0) , 'valor_total_da'] = 1

valor_tot_pago_ab_1['percentual_pago_cda'] = valor_tot_pago_ab_1['valor_pago'] / (valor_tot_pago_ab_1['valor_total_da'])

valor_tot_pago_ab_1

,cda,tipo_divida,id_pessoa,valor_total_da,valor_pago,valor_aberto,dif_tot_pago,dif_tot_pago_aberto,percentual_pago_cda
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,2048.73,0.0,2048.73,2048.73,0.0,0.0
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,2515.85,0.0,2515.85,2515.85,0.0,0.0
2,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,847.02,0.0,847.02,847.02,0.0,0.0
3,0000331f601a73e52b46f673bf0c61252,mercantil,e8424494daac9641,5385.81,0.0,5385.81,5385.81,0.0,0.0
4,00003d46e618da886b3a8e268c80aedf1,imovel,d8b23eda9800b9e3,6434.96,0.0,6434.96,6434.96,0.0,0.0
...,...,...,...,...,...,...,...,...,...
967074,ffffca43e897bbf0d8a3e5cebc255ca61,imovel,cad70f5b8701af03,5082.97,0.0,5082.97,5082.97,0.0,0.0
967075,ffffca43e897bbf0d8a3e5cebc255ca62,mercantil,e750ab1d88feb1ba,7634.36,0.0,7634.36,7634.36,0.0,0.0
967076,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,cc5afb41f6fbc367,1080.38,0.0,1080.38,1080.38,0.0,0.0
967077,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,9390.43,0.0,9390.43,9390.43,0.0,0.0


# CDAs com as quais vamos treinar (da_aberto == 0)

In [29]:
base_conj_da_ab_0_aux = bs_conj_da_ab_0[['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao', 'protesto',  'idade_divida', 'da_aberto']]

base_conj_da_ab_1_aux = bs_conj_da_ab_1[['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao', 'protesto',  'idade_divida', 'da_aberto']]

In [30]:
base_conj_da_ab_0_aux.shape[0]

1609354

In [31]:
base_conj_da_ab_1_aux.shape[0]

967079

In [32]:
print("Gerando variáveis para identificação dos grupos de contribuintes com instâncias de da_aberto == 0")


valor_tot_pago_ab_0 = pd.merge(base_conj_da_ab_0_aux, valor_tot_pago_ab_0, on = ['cda', 'tipo_divida', 'id_pessoa'], how = 'left')

valor_tot_pago_ab_0 = valor_tot_pago_ab_0[valor_tot_pago_ab_0['da_aberto'] == 0]

valor_tot_pago_ab_0.dropna(subset=['id_pessoa'], inplace=True)

valor_tot_pago_ab_0.loc[valor_tot_pago_ab_0['percentual_pago_cda'] > 1, 'percentual_pago_cda'] = 1

valor_tot_pago_ab_0

Gerando variáveis para identificação dos grupos de contribuintes com instâncias de da_aberto == 0


,cda,tipo_divida,id_pessoa,atividade_principal,situacao,protesto,idade_divida,da_aberto,valor_total_da,valor_pago,valor_aberto,dif_tot_pago,dif_tot_pago_aberto,percentual_pago_cda
0,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,INAPTO,0,21.0,0,1278.97,0.00,NaN,1278.97,NaN,0.000000
1,0000331f601a73e52b46f673bf0c61251,imovel,870c08c252b25ad1,APARTAMENTO,ATIVO,0,11.0,0,344.12,0.00,NaN,344.12,NaN,0.000000
2,000040eda866e3d19ceed47545e621dd1,imovel,9f3bac8718dac1fa,LOJA,ATIVO,0,15.0,0,10779.87,5438.40,NaN,5341.47,NaN,0.504496
3,000040eda866e3d19ceed47545e621dd2,mercantil,2f8ae1b9606267b4,INSTRUTOR DE TREINAMENTOS,INAPTO,0,15.0,0,328.63,0.00,NaN,328.63,NaN,0.000000
4,0000521b64bf28c8e52281a70553db461,imovel,b5df816e1786f0a3,CASA,ATIVO,0,7.0,0,490.83,0.00,NaN,490.83,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609349,ffffd79d8aa612ac9fcacff532cf50c41,imovel,c6b16a1235d987d3,APARTAMENTO,ATIVO,1,1.0,0,26104.42,26104.42,NaN,0.00,NaN,1.000000
1609350,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,cc5afb41f6fbc367,CONSULTORIA EM TECNOLOGIA DA INFORMAÇÃO,ATIVO,0,1.0,0,1431.77,265.15,NaN,1166.62,NaN,0.185190
1609351,ffffd8af5760cada9fcacff532cf50c41,imovel,d411f54ad97d1f39,NaN,ATIVO,0,4.0,0,1728.13,0.00,NaN,1728.13,NaN,0.000000
1609352,ffffdb721b90282718565e26094f06ee2,mercantil,51d6043a4efa1ed1,PUBLICITARIO (NÍVEL MÉDIO),ATIVO,0,6.0,0,240.77,0.00,NaN,240.77,NaN,0.000000


In [33]:
valor_tot_pago_ab_0.shape[0]

1609354

# CDAs que vamos predizer (da_aberto == 1)

In [34]:
valor_tot_pago_ab_1.shape[0]

967079

In [35]:
print("Gerando variáveis para identificação dos grupos de contribuintes com instâncias de da_aberto == 1")

valor_tot_pago_ab_1 = pd.merge(base_conj_da_ab_1_aux, valor_tot_pago_ab_1, on = ['cda', 'tipo_divida', 'id_pessoa'], how = 'left')

valor_tot_pago_ab_1 = valor_tot_pago_ab_1[valor_tot_pago_ab_1['da_aberto'] == 1]

valor_tot_pago_ab_1.dropna(subset=['id_pessoa'], inplace=True)

valor_tot_pago_ab_1.loc[valor_tot_pago_ab_1['percentual_pago_cda'] > 1, 'percentual_pago_cda'] = 1

valor_tot_pago_ab_1

Gerando variáveis para identificação dos grupos de contribuintes com instâncias de da_aberto == 1


,cda,tipo_divida,id_pessoa,atividade_principal,situacao,protesto,idade_divida,da_aberto,valor_total_da,valor_pago,valor_aberto,dif_tot_pago,dif_tot_pago_aberto,percentual_pago_cda
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,APARTAMENTO,ATIVO,0,1.0,1,2048.73,0.0,2048.73,2048.73,0.0,0.0
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,ATIVIDADES DE TELEATENDIMENTO,SUSPENSO,0,1.0,1,2515.85,0.0,2515.85,2515.85,0.0,0.0
2,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,Comercio varejista especializado de equipament...,ATIVO,0,0.0,1,847.02,0.0,847.02,847.02,0.0,0.0
3,0000331f601a73e52b46f673bf0c61252,mercantil,e8424494daac9641,"COM VAR DE MERC EM GERAL, COM PREDOM DE PROD A...",INAPTO,0,11.0,1,5385.81,0.0,5385.81,5385.81,0.0,0.0
4,00003d46e618da886b3a8e268c80aedf1,imovel,d8b23eda9800b9e3,CASA,EM PROCESSO DE BAIXA,0,12.0,1,6434.96,0.0,6434.96,6434.96,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967074,ffffca43e897bbf0d8a3e5cebc255ca61,imovel,cad70f5b8701af03,CASA,ATIVO,0,11.0,1,5082.97,0.0,5082.97,5082.97,0.0,0.0
967075,ffffca43e897bbf0d8a3e5cebc255ca62,mercantil,e750ab1d88feb1ba,ATIVIDADES DE ASSOCIAÇÕES DE DEFESA DE DIREITO...,ATIVO,0,11.0,1,7634.36,0.0,7634.36,7634.36,0.0,0.0
967076,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,cc5afb41f6fbc367,CONSULTORIA EM TECNOLOGIA DA INFORMAÇÃO,ATIVO,0,1.0,1,1080.38,0.0,1080.38,1080.38,0.0,0.0
967077,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,APARTAMENTO,ATIVO,0,25.0,1,9390.43,0.0,9390.43,9390.43,0.0,0.0


In [36]:
valor_tot_pago_ab_1.shape[0]

967079

### Dados da DÍVIDA

In [ ]:
# # Seleciona dados sobre a divida
# dados_divida = reconstroi[[ 'cda', 'id_pessoa', 'tipo_divida', 'valor_tot', 'valor_pago', 'protesto', 'ajuizamento', 'idade_divida', 'da_aberto']]


# dados_divida.dropna(subset=['id_pessoa'], inplace=True)

### Variável Resposta Y: percentual_pago_cda

In [37]:
# Imputando historico_pagamento_em_valor = 1 nos casos que passa de 1
# ESCOLHA DA LARISSA
valor_tot_pago_ab_0['percentual_pago_cda'].max()
#valor_tot_pago_ab_1.loc[valor_tot_pago_ab_1['percentual_pago_cda'] > 1, 'percentual_pago_cda'] = 1

1.0

# Analisando

In [38]:
def contar_percentual_pago_cda_faixas(df_divida_ativa):

    data_percentil = {
        'inf': [0, 0.01, 0.25, 0.5, 0.75, 0.99, 1],
        'sup': [0.01, 0.25, 0.5, 0.75, 0.99, 1, 1.01]
    }

    # Creating a DataFrame from the dictionary
    df_cut_percentil = pd.DataFrame(data_percentil)

    df_cut_percentil['intervalo'] = df_cut_percentil.apply(paste_intervalo, axis = 1)

    # Define the bins and labels for groups
    igr_bins = [0, 0.01, 0.25, 0.5, 0.75, 0.99, 1, 1.01]
    igr_labels = df_cut_percentil['intervalo']

    df_divida_ativa['percentual_pago_cda_faixas'] = pd.cut(df_divida_ativa['percentual_pago_cda'], 
                                    bins = igr_bins, 
                                    labels = igr_labels, 
                                    right = False)
    
    return df_divida_ativa

In [39]:
valor_tot_pago_ab_0_cont_faixas = contar_percentual_pago_cda_faixas(valor_tot_pago_ab_0)
valor_tot_pago_ab_1_cont_faixas = contar_percentual_pago_cda_faixas(valor_tot_pago_ab_1)

In [40]:
def freq_percentual_pago_cda_faixas(df_divida_ativa):

    df_metrics = df_divida_ativa.groupby('percentual_pago_cda_faixas')['percentual_pago_cda_faixas'].count().to_frame() 
    total = df_metrics['percentual_pago_cda_faixas'].sum()   
    df_metrics['total'] = total
    df_metrics['perc'] = df_metrics['percentual_pago_cda_faixas']/total
    return df_metrics

In [41]:
valor_tot_pago_ab_0_cont_faixas = freq_percentual_pago_cda_faixas(valor_tot_pago_ab_0_cont_faixas)
valor_tot_pago_ab_0_cont_faixas

,percentual_pago_cda_faixas,total,perc
percentual_pago_cda_faixas,,,
"[0.0, 0.01)",994029,1609354,0.617657
"[0.01, 0.25)",157574,1609354,0.097911
"[0.25, 0.5)",101210,1609354,0.062889
"[0.5, 0.75)",57879,1609354,0.035964
"[0.75, 0.99)",2788,1609354,0.001732
"[0.99, 1.0)",3,1609354,0.000002
"[1.0, 1.01)",295871,1609354,0.183845


In [42]:
valor_tot_pago_ab_1_cont_faixas = freq_percentual_pago_cda_faixas(valor_tot_pago_ab_1_cont_faixas)
valor_tot_pago_ab_1_cont_faixas

,percentual_pago_cda_faixas,total,perc
percentual_pago_cda_faixas,,,
"[0.0, 0.01)",967059,967079,0.999979
"[0.01, 0.25)",2,967079,0.000002
"[0.25, 0.5)",4,967079,0.000004
"[0.5, 0.75)",5,967079,0.000005
"[0.75, 0.99)",8,967079,0.000008
"[0.99, 1.0)",1,967079,0.000001
"[1.0, 1.01)",0,967079,0.000000


In [ ]:
# df_metrics_aberto = df_divida_ativa.groupby(['da_aberto', 'percentual_pago_cda_faixas'])['percentual_pago_cda_faixas'].count().to_frame()
# df_metrics_aberto = df_metrics_aberto.rename(columns = {'percentual_pago_cda_faixas':'cda'})
# df_metrics_aberto = df_metrics_aberto.reset_index()

# total = df_metrics_aberto.groupby(['da_aberto'])['cda'].sum().to_frame().reset_index()
# total = total.rename(columns = {'cda':'total'})
# df_metrics_aberto = pd.merge(df_metrics_aberto, total, on = 'da_aberto')


# df_metrics_aberto['perc'] = df_metrics_aberto['cda']/df_metrics_aberto['total']
# df_metrics_aberto

In [ ]:
valor_tot_pago_ab_0.columns

# Existem CDAs totalmente pagas emm aberto!

# Salvar no S3

In [43]:
print("Inicia a conexão com S3 para inscrição dos dados")
# Cria conexão ao s3 e preenche a tabela com os dados
s3_resource = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACESS_KEY")
    )

NOME_ARQ_SALVAR_S3 = 'feature_store_igr_prime_ab0_maxpg_lari.csv'

up_s3_files(dataframe=valor_tot_pago_ab_0, 
            bucket_name=os.getenv("S3_BUCKET_NAME"), 
            folder_name=os.getenv("S3_FOLDER_NAME"), 
            file_name=NOME_ARQ_SALVAR_S3)


NOME_ARQ_SALVAR_S3 = 'feature_store_igr_prime_ab1_maxpg_lari.csv'

up_s3_files(dataframe=valor_tot_pago_ab_1, 
            bucket_name=os.getenv("S3_BUCKET_NAME"), 
            folder_name=os.getenv("S3_FOLDER_NAME"), 
            file_name=NOME_ARQ_SALVAR_S3)


print("Dados upados no s3")
print("Processo finalizado")

Inicia a conexão com S3 para inscrição dos dados
Dados upados no s3
Processo finalizado
